In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.fft import fft 

## Data Processing

In [2]:
# Load and preprocess data
df = pd.read_csv('../22_HP/outputs/simulation_results_nand.csv')
df = df.drop(columns=['delay_LH_NodeA','delay_HL_NodeA','delay_LH_NodeB','delay_HL_NodeB'], axis=1)

In [3]:
df.head()

,Vin_A,Vin_B,temp,pvdd,cqload,lmin,wmin,toxe_n,toxm_n,toxref_n,toxe_p,toxm_p,toxref_p,toxp_par,xj_n,xj_p,ndep_n,ndep_p,leakage
0,0.0,0.0,43.786431,1.049654,1.992558e-16,4.422860e-08,4.335549e-08,8.858098e-10,9.149308e-10,8.800085e-10,9.382331e-10,9.149568e-10,9.596845e-10,6.400032e-10,1.438907e-08,1.434486e-08,6.530156e+18,2.751990e+18,1.136609e-08
1,0.0,1.0,43.786431,1.049654,1.992558e-16,4.422860e-08,4.335549e-08,8.858098e-10,9.149308e-10,8.800085e-10,9.382331e-10,9.149568e-10,9.596845e-10,6.400032e-10,1.438907e-08,1.434486e-08,6.530156e+18,2.751990e+18,1.114162e-08
2,1.0,0.0,43.786431,1.049654,1.992558e-16,4.422860e-08,4.335549e-08,8.858098e-10,9.149308e-10,8.800085e-10,9.382331e-10,9.149568e-10,9.596845e-10,6.400032e-10,1.438907e-08,1.434486e-08,6.530156e+18,2.751990e+18,6.543595e-09
3,1.0,1.0,43.786431,1.049654,1.992558e-16,4.422860e-08,4.335549e-08,8.858098e-10,9.149308e-10,8.800085e-10,9.382331e-10,9.149568e-10,9.596845e-10,6.400032e-10,1.438907e-08,1.434486e-08,6.530156e+18,2.751990e+18,1.866817e-08
4,0.0,0.0,73.734086,0.936041,1.703362e-15,4.468047e-08,4.295989e-08,8.966653e-10,9.579161e-10,8.894931e-10,9.051161e-10,8.974949e-10,9.112459e-10,6.536598e-10,1.335243e-08,1.435760e-08,6.696820e+18,2.698271e+18,9.145988e-09


In [4]:
df.describe()

,Vin_A,Vin_B,temp,pvdd,cqload,lmin,wmin,toxe_n,toxm_n,toxref_n,toxe_p,toxm_p,toxref_p,toxp_par,xj_n,xj_p,ndep_n,ndep_p,leakage
count,40000.000000,40000.000000,40000.000000,40000.000000,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04,4.000000e+04
mean,0.500000,0.500000,34.362600,0.999048,2.511981e-15,4.503877e-08,4.500441e-08,9.002687e-10,8.996303e-10,9.003235e-10,9.200230e-10,9.202474e-10,9.208007e-10,6.500293e-10,1.399542e-08,1.401062e-08,6.498297e+18,2.799952e+18,1.272208e-08
std,0.500006,0.500006,52.127050,0.058259,1.435116e-15,1.493685e-09,1.485418e-09,2.972510e-11,3.026050e-11,2.993862e-11,3.073110e-11,3.071142e-11,3.079464e-11,2.175375e-11,4.665974e-10,4.640081e-10,2.137011e+17,9.311797e+16,1.107019e-08
min,0.000000,0.000000,-54.986959,0.900033,1.035900e-17,4.015305e-08,3.938417e-08,7.666010e-10,7.602014e-10,7.868514e-10,7.979490e-10,8.003992e-10,8.051369e-10,5.724874e-10,1.173568e-08,1.244580e-08,5.739411e+18,2.435456e+18,4.540209e-10
25%,0.000000,0.000000,-10.844191,0.948592,1.256067e-15,4.403489e-08,4.397007e-08,8.805222e-10,8.795919e-10,8.801323e-10,8.994353e-10,8.997851e-10,9.000814e-10,6.354986e-10,1.368186e-08,1.369309e-08,6.352741e+18,2.736722e+18,6.125763e-09
50%,0.500000,0.500000,33.830185,0.998653,2.520102e-15,4.504623e-08,4.502020e-08,9.004051e-10,8.998323e-10,9.005850e-10,9.199003e-10,9.201744e-10,9.208642e-10,6.498078e-10,1.399949e-08,1.400450e-08,6.497865e+18,2.801193e+18,9.706318e-09
75%,1.000000,1.000000,79.692719,1.049947,3.755893e-15,4.603464e-08,4.600447e-08,9.199819e-10,9.201116e-10,9.205995e-10,9.404933e-10,9.408495e-10,9.417332e-10,6.647795e-10,1.431343e-08,1.431982e-08,6.641405e+18,2.862151e+18,1.542349e-08
max,1.000000,1.000000,124.996031,1.099992,4.999631e-15,5.073782e-08,5.021168e-08,1.014954e-09,1.014115e-09,1.011088e-09,1.032195e-09,1.033069e-09,1.043273e-09,7.428602e-10,1.579761e-08,1.582100e-08,7.419051e+18,3.151342e+18,1.283613e-07


In [5]:
df.dtypes

Vin_A       float64
Vin_B       float64
temp        float64
pvdd        float64
cqload      float64
lmin        float64
wmin        float64
toxe_n      float64
toxm_n      float64
toxref_n    float64
toxe_p      float64
toxm_p      float64
toxref_p    float64
toxp_par    float64
xj_n        float64
xj_p        float64
ndep_n      float64
ndep_p      float64
leakage     float64
dtype: object

In [6]:
# split into input (X) and output (y) variables
x = df.drop(columns=['leakage'], axis=1).values
y = df['leakage'].values

## Training and Testing

In [7]:
def train_and_test(model, x, y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
  model.fit(x_train, y_train)
  y_pred = model.predict(x_test)
  mse = mean_squared_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)
  return mse, r2

**Non-linear Relationships:** The dataset may contain non-linear relationships between input features
and target variables (leakage and delay) with influencing factors like temperature, voltage, and load conditions. 

**Continuous Target Variable:** Leakage power is a continuous value, making regression models suitable for prediction.

**Robustness**: They are robust to outliers and noise in the data, which might be present in real-world
datasets like the one under consideration.

Gradient Boosting models excel at capturing these non-linear relationships by building ensembles of weak decision trees, progressively improving predictions with each iteration.


### Linear Regression

In [8]:
def train_and_test_linear_reg(x, y):
  
  model = LinearRegression()
  return train_and_test(model, x, y)

### Adaboost

In [9]:
from sklearn.ensemble import AdaBoostRegressor

def train_and_test_adaboost(x, y):

  model = AdaBoostRegressor(random_state=42)
  return train_and_test(model, x, y)

### XGBoost

In [111]:
import xgboost as xgb

def train_and_test_xgboost(x, y):
      
    model = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                  max_depth = 5, alpha = 10, n_estimators = 10)
    return train_and_test(model, x, y)

mse_xg, r2_xg = train_and_test_xgboost(x, y)
print("\nXGBoost Model Performance:")
print("Mean Squared Error:", mse_xg)
print("R2 Score:", r2_xg)


XGBoost Model Performance:
Mean Squared Error: 1.230189438563328e-16
R2 Score: -1.8423294894098774e-06


### LightGBM

In [62]:

import lightgbm as lgb

def train_and_test_lightgbm(x, y):
        
     model = lgb.LGBMRegressor()
     return train_and_test(model, x, y)


### Cat Boost Regressor

In [11]:
from catboost import CatBoostRegressor

def train_and_test_catboost(x, y):
    
  model = CatBoostRegressor(random_state=42) 
  return train_and_test(model, x, y)

### Random Forest Regressor

In [112]:
from sklearn.ensemble import RandomForestRegressor

def train_and_test_random_forest(x, y):
     
      model = RandomForestRegressor(random_state=42)
      return train_and_test(model, x, y)

mse_rf, r2_rf = train_and_test_random_forest(x, y)
print("\nRandom Forest Model Performance:")
print("Mean Squared Error:", mse_rf)
print("R2 Score:", r2_rf)


Random Forest Model Performance:
Mean Squared Error: 1.2301893370627395e-16
R2 Score: -1.7598212411762404e-06


In [113]:
mse_lr, r2_lr = train_and_test_linear_reg(x, y)
mse_ada, r2_ada = train_and_test_adaboost(x, y)
mse_lgb, r2_lgb = train_and_test_lightgbm(x, y)
mse_catboost, r2_catboost = train_and_test_catboost(x, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001849 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4084
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 2.92ms	remaining: 2.91s
1:	learn: 0.0000000	total: 5.58ms	remaining: 2.79s
2:	learn: 0.0000000	total: 10.1ms	remaining: 3.37s
3:	learn: 0.0000000	total: 15.5ms	remaining: 3.87s
4:	learn: 0.0000000	total: 20.7ms	remaining: 4.12s
5:	learn: 0.0000000	total: 25.6ms	remaining: 4.25s
6:	learn: 0.0000000	total: 29.9ms	remaining: 4.24s
7:	learn: 0.0000000	total: 33.2ms	remaining: 4.11s
8:	learn: 0.0000000	total: 38.5ms	remaining: 4.24s
9:	learn: 0.0000000	total: 42.6ms	remaining: 4.21s
10:	learn: 0.0000000	total: 45.6ms	remaining: 4.1s
11:	learn: 0.0000000	total: 48.4ms	remaining: 3.99s
12:	learn: 0.00

In [116]:
print("Model Performance Before Transformation\n")

print("Linear Regression Model Performance:")
print("Mean Squared Error:", mse_lr)
print("R2 Score:", r2_lr)

print("\nAdaBoost Model Performance:")
print("Mean Squared Error:", mse_ada)
print("R2 Score:", r2_ada)

print("\nLightGBM Model Performance:")
print("Mean Squared Error:", mse_lgb)
print("R2 Score:", r2_lgb)

print("\nCatBoost Model Performance:")
print("Mean Squared Error:", mse_catboost)
print("R2 Score:", r2_catboost)


Model Performance Before Transformation

Linear Regression Model Performance:
Mean Squared Error: 1.0152560995228213e-16
R2 Score: 0.17471412277385678

AdaBoost Model Performance:
Mean Squared Error: 1.3638890430300284e-16
R2 Score: -0.10868416929009572

LightGBM Model Performance:
Mean Squared Error: 2.2366763187613596e-18
R2 Score: 0.9818184064230937

CatBoost Model Performance:
Mean Squared Error: 5.269388109071569e-19
R2 Score: 0.995716596442923


## Transformations

### FFT

In [66]:
def apply_fft(x):
  x_transformed = np.zeros_like(x)
  for i, row in enumerate(x):
    x_transformed[i] = np.abs(fft(row))  
  return x_transformed

# Apply FFT to the input data
x_fft = apply_fft(x)
y_fft = apply_fft(y.reshape(-1, 1))

In [67]:
mse_lr_fft, r2_lr_fft = train_and_test_linear_reg(x_fft, y)
mse_ada_fft, r2_ada_fft = train_and_test_adaboost(x_fft, y)
mse_lgb_fft, r2_lgb_fft = train_and_test_lightgbm(x_fft, y)
mse_catboost_fft, r2_catboost_fft = train_and_test_catboost(x_fft, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001645 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4590
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 4.63ms	remaining: 4.62s
1:	learn: 0.0000000	total: 11.2ms	remaining: 5.6s
2:	learn: 0.0000000	total: 15.2ms	remaining: 5.06s
3:	learn: 0.0000000	total: 18.3ms	remaining: 4.57s
4:	learn: 0.0000000	total: 21.2ms	remaining: 4.22s
5:	learn: 0.0000000	total: 24.1ms	remaining: 3.99s
6:	learn: 0.0000000	total: 26.5ms	remaining: 3.76s
7:	learn: 0.0000000	total: 29.9ms	remaining: 3.7s
8:	learn: 0.0000000	total: 33.9ms	remaining: 3.73s
9:	learn: 0.0000000	total: 36.9ms	remaining: 3.65s
10:	learn: 0.0000000	total: 39.4ms	remaining: 3.55s
11:	learn: 0.0000000	total: 42.1ms	remaining: 3.46s
12:	learn: 0.000

In [70]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_fft)
print("  R2 Score:", r2_lr_fft)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_fft)
print("  R2 Score:", r2_ada_fft)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_fft)
print("  R2 Score:", r2_lgb_fft)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_fft)
print("  R2 Score:", r2_catboost_fft)




Linear Regression:
  Mean Squared Error: 1.230752112750034e-16
  R2 Score: -0.00045923141583559257

AdaBoost:
  Mean Squared Error: 2.083279917661038e-16
  R2 Score: -0.6934658113973242

LightGBM:
  Mean Squared Error: 1.1559793105853377e-16
  R2 Score: 0.06032241535895555

CatBoost:
  Mean Squared Error: 1.1175821488081677e-16
  R2 Score: 0.09153487037908281


In [71]:
# square root of the sum of the squares of the real and imaginary parts of the FFT
def fft_magnitude(x):
    return np.sqrt(np.sum(np.abs(fft(x))**2))

x_scaled = scaler.fit_transform(df.drop('leakage', axis=1).values)
# apply the FFT to each row of the input data
x_fft = np.apply_along_axis(fft_magnitude, 1, x_scaled)


In [72]:
mse_lr_fft, r2_lr_fft = train_and_test_linear_reg(x_fft.reshape(-1, 1), y)
mse_ada_fft, r2_ada_fft = train_and_test_adaboost(x_fft.reshape(-1, 1), y)
mse_lgb_fft, r2_lgb_fft = train_and_test_lightgbm(x_fft.reshape(-1, 1), y)
mse_catboost_fft, r2_catboost_fft = train_and_test_catboost(x_fft.reshape(-1, 1), y)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000059 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 1
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 2.51ms	remaining: 2.5s
1:	learn: 0.0000000	total: 4.32ms	remaining: 2.15s
2:	learn: 0.0000000	total: 5.98ms	remaining: 1.99s
3:	learn: 0.0000000	total: 7.63ms	remaining: 1.9s
4:	learn: 0.0000000	total: 9.29ms	remaining: 1.85s
5:	learn: 0.0000000	total: 11ms	remaining: 1.82s
6:	learn: 0.0000000	total: 12.8ms	remaining: 1.82s
7:	learn: 0.0000000	total: 15.1ms	remaining: 1.87s
8:	learn: 0.0000000	total: 17.4ms	remaining: 1.92s
9:	learn: 0.0000000	total: 20.5ms	remaining: 2.03s
10:	learn: 0.0000000	total: 23.4ms	remaining: 2.1s
11:	learn

In [73]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_fft)
print("  R2 Score:", r2_lr_fft)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_fft)
print("  R2 Score:", r2_ada_fft)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_fft)
print("  R2 Score:", r2_lgb_fft)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_fft)
print("  R2 Score:", r2_catboost_fft)

Linear Regression:
  Mean Squared Error: 1.2202793894170019e-16
  R2 Score: 0.008053882336359952

AdaBoost:
  Mean Squared Error: 2.2133354654511863e-16
  R2 Score: -0.799185941418278

LightGBM:
  Mean Squared Error: 1.2146094338284297e-16
  R2 Score: 0.012662900961263879

CatBoost:
  Mean Squared Error: 1.216969468671236e-16
  R2 Score: 0.010744465379891599


### PCA

In [103]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.95)  
x_pca = pca.fit_transform(x_scaled)

In [104]:
mse_lr_pca, r2_lr_pca = train_and_test_linear_reg(x_pca, y)
mse_ada_pca, r2_ada_pca = train_and_test_adaboost(x_pca, y)
mse_lgb_pca, r2_lgb_pca = train_and_test_lightgbm(x_pca, y)
mse_catboost_pca, r2_catboost_pca = train_and_test_catboost(x_pca, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001403 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4338
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 7.82ms	remaining: 7.81s
1:	learn: 0.0000000	total: 15.2ms	remaining: 7.56s
2:	learn: 0.0000000	total: 19.3ms	remaining: 6.42s
3:	learn: 0.0000000	total: 24.1ms	remaining: 6.01s
4:	learn: 0.0000000	total: 29.5ms	remaining: 5.88s
5:	learn: 0.0000000	total: 33.6ms	remaining: 5.56s
6:	learn: 0.0000000	total: 37.5ms	remaining: 5.32s
7:	learn: 0.0000000	total: 41.6ms	remaining: 5.16s
8:	learn: 0.0000000	total: 47.6ms	remaining: 5.24s
9:	learn: 0.0000000	total: 50.4ms	remaining: 4.99s
10:	learn: 0.0000000	total: 54ms	remaining: 4.86s
11:	learn: 0.0000000	total: 57.9ms	remaining: 4.77s
12:	learn: 0.000

In [105]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_pca)
print("  R2 Score:", r2_lr_pca)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_pca)
print("  R2 Score:", r2_ada_pca)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_pca)
print("  R2 Score:", r2_lgb_pca)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_pca)
print("  R2 Score:", r2_catboost_pca)


Linear Regression:
  Mean Squared Error: 5.759237526350362e-17
  R2 Score: 0.531840547786737

AdaBoost:
  Mean Squared Error: 1.5266108794433766e-16
  R2 Score: -0.24095821676575957

LightGBM:
  Mean Squared Error: 1.7609384616289533e-17
  R2 Score: 0.8568560539818714

CatBoost:
  Mean Squared Error: 1.1246723857309819e-17
  R2 Score: 0.908577132716931


### Quantile Transformer

This transformation scales features to a specific range (often 0-1) based on quantiles. It preserves the distribution of the data more closely compared to scaling techniques like StandardScaler.

Applications: Useful for models sensitive to feature scaling or when you want to ensure features fall within a specific range. It can also be helpful for dealing with skewed data distributions.

In [77]:
from sklearn.preprocessing import QuantileTransformer

transformer = QuantileTransformer(output_distribution='uniform')  
x_qt = transformer.fit_transform(x)

In [75]:
mse_lr_qt, r2_lr_qt = train_and_test_linear_reg(x_qt, y)
mse_ada_qt, r2_ada_qt = train_and_test_adaboost(x_qt, y)
mse_lgb_qt, r2_lgb_qt = train_and_test_lightgbm(x_qt, y)
mse_catboost_qt, r2_catboost_qt = train_and_test_catboost(x_qt, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001575 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4084
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 4.18ms	remaining: 4.17s
1:	learn: 0.0000000	total: 7.39ms	remaining: 3.69s
2:	learn: 0.0000000	total: 11.3ms	remaining: 3.76s
3:	learn: 0.0000000	total: 18ms	remaining: 4.48s
4:	learn: 0.0000000	total: 21.5ms	remaining: 4.28s
5:	learn: 0.0000000	total: 27.4ms	remaining: 4.54s
6:	learn: 0.0000000	total: 32.3ms	remaining: 4.58s
7:	learn: 0.0000000	total: 35.5ms	remaining: 4.4s
8:	learn: 0.0000000	total: 38.7ms	remaining: 4.27s
9:	learn: 0.0000000	total: 46.6ms	remaining: 4.61s
10:	learn: 0.0000000	total: 50.6ms	remaining: 4.55s
11:	learn: 0.0000000	total: 53.8ms	remaining: 4.43s
12:	learn: 0.0000

In [76]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_qt)
print("  R2 Score:", r2_lr_qt)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_qt)
print("  R2 Score:", r2_ada_qt)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_qt)
print("  R2 Score:", r2_lgb_qt)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_qt)
print("  R2 Score:", r2_catboost_qt)

Linear Regression:
  Mean Squared Error: 5.889349656702298e-17
  R2 Score: 0.5212639352762847

AdaBoost:
  Mean Squared Error: 1.170433628679323e-16
  R2 Score: 0.04857272521327971

LightGBM:
  Mean Squared Error: 2.2042061952888595e-18
  R2 Score: 0.9820823510016709

CatBoost:
  Mean Squared Error: 5.201129065305277e-19
  R2 Score: 0.9957720831569056


### Scaling

StandardScaler removes the mean and scales each feature/variable to unit variance. Scaled values will be close to 1 and -1.

In [79]:
# Standardize the data
scaler = StandardScaler()
x_scaled = scaler.fit_transform(df.drop('leakage', axis=1).values)
y_scaled = scaler.fit_transform(df['leakage'].values.reshape(-1, 1))
x_scaled

array([[-1.        , -1.        ,  0.18078805, ...,  0.72033654,
         0.14908407, -0.5150728 ],
       [-1.        ,  1.        ,  0.18078805, ...,  0.72033654,
         0.14908407, -0.5150728 ],
       [ 1.        , -1.        ,  0.18078805, ...,  0.72033654,
         0.14908407, -0.5150728 ],
       ...,
       [-1.        ,  1.        ,  1.0950481 , ...,  1.6576675 ,
        -1.23959102,  1.93594467],
       [ 1.        , -1.        ,  1.0950481 , ...,  1.6576675 ,
        -1.23959102,  1.93594467],
       [ 1.        ,  1.        ,  1.0950481 , ...,  1.6576675 ,
        -1.23959102,  1.93594467]])

In [80]:
mse_lr_ss, r2_lr_ss = train_and_test_linear_reg(x_scaled, y)
mse_ada_ss, r2_ada_ss = train_and_test_adaboost(x_scaled, y)
mse_lgb_ss, r2_lgb_ss = train_and_test_xgboost(x_scaled, y)
mse_catboost_ss, r2_catboost_ss = train_and_test_catboost(x_scaled, y)

Learning rate set to 0.069316
0:	learn: 0.0000000	total: 4.1ms	remaining: 4.1s
1:	learn: 0.0000000	total: 7.02ms	remaining: 3.5s
2:	learn: 0.0000000	total: 11.9ms	remaining: 3.95s
3:	learn: 0.0000000	total: 15.7ms	remaining: 3.92s
4:	learn: 0.0000000	total: 20.3ms	remaining: 4.04s
5:	learn: 0.0000000	total: 22.8ms	remaining: 3.77s
6:	learn: 0.0000000	total: 28.6ms	remaining: 4.06s
7:	learn: 0.0000000	total: 32.7ms	remaining: 4.06s
8:	learn: 0.0000000	total: 36.8ms	remaining: 4.05s
9:	learn: 0.0000000	total: 39.5ms	remaining: 3.91s
10:	learn: 0.0000000	total: 45.5ms	remaining: 4.09s
11:	learn: 0.0000000	total: 50.8ms	remaining: 4.18s
12:	learn: 0.0000000	total: 53.8ms	remaining: 4.08s
13:	learn: 0.0000000	total: 57.5ms	remaining: 4.05s
14:	learn: 0.0000000	total: 61.7ms	remaining: 4.05s
15:	learn: 0.0000000	total: 68.3ms	remaining: 4.2s
16:	learn: 0.0000000	total: 72.1ms	remaining: 4.17s
17:	learn: 0.0000000	total: 76.8ms	remaining: 4.19s
18:	learn: 0.0000000	total: 82.1ms	remaining: 4.

In [81]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_ss)
print("  R2 Score:", r2_lr_ss)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_ss)
print("  R2 Score:", r2_ada_ss)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_ss)
print("  R2 Score:", r2_lgb_ss)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_ss)
print("  R2 Score:", r2_catboost_ss)


Linear Regression:
  Mean Squared Error: 5.759237526350362e-17
  R2 Score: 0.531840547786737

AdaBoost:
  Mean Squared Error: 1.0392358876195854e-16
  R2 Score: 0.1552213263607778

LightGBM:
  Mean Squared Error: 1.230189438563328e-16
  R2 Score: -1.8423294894098774e-06

CatBoost:
  Mean Squared Error: 5.201129065305277e-19
  R2 Score: 0.9957720831569056


### Yeo johnson transformation

Yeo Johnson’s transformations technique can be applied to zero and negative values as well. So we apply this on scaled data. Advanced form of box cox transformations technique - boxcox cannot be applied because 0 can also be one of the values in the dataset, but only possible values are allowed for this transformation.

In [82]:
# yeo johnson transformation
from scipy.stats import yeojohnson

x_yeo = np.zeros_like(x_scaled)
for i in range(x_scaled.shape[1]):
    x_yeo[:, i], _ = yeojohnson(x_scaled[:, i]) 


In [83]:
mse_lr_yeo, r2_lr_yeo = train_and_test_linear_reg(x_yeo, y)
mse_ada_yeo, r2_ada_yeo = train_and_test_adaboost(x_yeo, y)
mse_lgb_yeo, r2_lgb_yeo = train_and_test_lightgbm(x_yeo, y)
mse_catboost_yeo, r2_catboost_yeo = train_and_test_catboost(x_yeo, y)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4086
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 2.88ms	remaining: 2.88s
1:	learn: 0.0000000	total: 5.42ms	remaining: 2.71s
2:	learn: 0.0000000	total: 9.96ms	remaining: 3.31s
3:	learn: 0.0000000	total: 12.9ms	remaining: 3.21s
4:	learn: 0.0000000	total: 16.7ms	remaining: 3.31s
5:	learn: 0.0000000	total: 19.8ms	remaining: 3.28s
6:	learn: 0.0000000	total: 22.4ms	remaining: 3.17s
7:	learn: 0.0000000	total: 25.1ms	remaining: 3.11s
8:	learn: 0.0000000	total: 27.4ms	remaining: 3.02s
9:	learn: 0.0000000	total: 29.6ms	remaining: 2.93s
10:	learn: 0.0000000	total: 32.9ms	remaining: 2.96s
11:	learn: 0.0000000	total: 36.3ms	remaining: 2.99s
12:	learn: 0.0

In [84]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_yeo)
print("  R2 Score:", r2_lr_yeo)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_yeo)
print("  R2 Score:", r2_ada_yeo)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_yeo)
print("  R2 Score:", r2_lgb_yeo)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_yeo)
print("  R2 Score:", r2_catboost_yeo)

Linear Regression:
  Mean Squared Error: 5.775241508932566e-17
  R2 Score: 0.5305396089585266

AdaBoost:
  Mean Squared Error: 1.0392358876195854e-16
  R2 Score: 0.1552213263607778

LightGBM:
  Mean Squared Error: 2.142238113212887e-18
  R2 Score: 0.9825860798933281

CatBoost:
  Mean Squared Error: 5.201129065305277e-19
  R2 Score: 0.9957720831569056


### Power Transformer

This transformation is similar to Yeo-Johnson but offers more flexibility and can handle a wider range of data distributions. It automatically finds the optimal lambda parameter for the transformation, which can be more convenient than manually tuning it in Yeo-Johnson.

In [85]:
from sklearn.preprocessing import PowerTransformer

transformer = PowerTransformer()
x_powt = transformer.fit_transform(x_scaled)


In [86]:
mse_lr_powt, r2_lr_powt = train_and_test_linear_reg(x_powt, y)
mse_ada_powt, r2_ada_powt = train_and_test_adaboost(x_powt, y)
mse_lgb_powt, r2_lgb_powt = train_and_test_lightgbm(x_powt, y)
mse_catboost_powt, r2_catboost_powt = train_and_test_catboost(x_powt, y)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001567 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4086
[LightGBM] [Info] Number of data points in the train set: 28000, number of used features: 18
[LightGBM] [Info] Start training from score 0.000000
Learning rate set to 0.069316
0:	learn: 0.0000000	total: 3.14ms	remaining: 3.13s
1:	learn: 0.0000000	total: 7.9ms	remaining: 3.94s
2:	learn: 0.0000000	total: 10.6ms	remaining: 3.54s
3:	learn: 0.0000000	total: 14.3ms	remaining: 3.55s
4:	learn: 0.0000000	total: 17.3ms	remaining: 3.44s
5:	learn: 0.0000000	total: 22.6ms	remaining: 3.74s
6:	learn: 0.0000000	total: 25.3ms	remaining: 3.58s
7:	learn: 0.0000000	total: 29.1ms	remaining: 3.61s
8:	learn: 0.0000000	total: 32.4ms	remaining: 3.56s
9:	learn: 0.0000000	total: 35.3ms	remaining: 3.49s
10:	learn: 0.0000000	total: 39.4ms	remaining: 3.54s
11:	learn: 0.0000000	total: 41.7ms	remaining: 3.43s
12:	learn: 0.00

In [87]:
print("Linear Regression:")
print("  Mean Squared Error:", mse_lr_powt)
print("  R2 Score:", r2_lr_powt)
print("\nAdaBoost:")
print("  Mean Squared Error:", mse_ada_powt)
print("  R2 Score:", r2_ada_powt)
print("\nLightGBM:")
print("  Mean Squared Error:", mse_lgb_powt)
print("  R2 Score:", r2_lgb_powt)
print("\nCatBoost:")
print("  Mean Squared Error:", mse_catboost_powt)
print("  R2 Score:", r2_catboost_powt)

Linear Regression:
  Mean Squared Error: 5.775241518665368e-17
  R2 Score: 0.5305396081673623

AdaBoost:
  Mean Squared Error: 1.0392358876195854e-16
  R2 Score: 0.1552213263607778

LightGBM:
  Mean Squared Error: 2.222869524059344e-18
  R2 Score: 0.9819306397077072

CatBoost:
  Mean Squared Error: 5.201129065305277e-19
  R2 Score: 0.9957720831569056
